In [3]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from opmd_viewer import OpenPMDTimeSeries

In [34]:
# ts = OpenPMDTimeSeries('/tmp/nbody_gpu_test_run')
ts = OpenPMDTimeSeries('/mnt/hdd/data/gpu_run_debug3')
# ts = OpenPMDTimeSeries('/home/dominik/Code/openPMD/openPMD-example-datasets/example-3d/hdf5/')
ts

In [35]:
# Interactive GUI
ts.slider()

FigureCanvasNbAgg()

In [36]:
ts.h5_files

['/mnt/hdd/data/gpu_run_debug3/data00000000.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000100.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000200.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000300.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000400.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000500.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000600.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000700.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000800.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00000900.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001000.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001100.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001200.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001300.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001400.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001500.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001600.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001700.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001800.h5',
 '/mnt/hdd/data/gpu_run_debug3/data00001900.h5',
 '/mnt/hdd/data/gpu_

In [39]:
import h5py
h5_files = [h5py.File(f) for f in ts.h5_files[:3]]

In [41]:
import pandas as pd
df = pd.read_json('/mnt/hdd/data/gpu_run_debug3/diagnostic_results.json').T
df["total_energy"] = df["kinetic_energy"] + df["potential_energy"]
df

,kinetic_energy,potential_energy,temperature,total_energy
0,1.606185e+46,-2.165433e-195,1.517750e+66,1.606185e+46
100,1.606185e+46,-2.039936e-207,1.517750e+66,1.606185e+46
200,1.606185e+46,-3.283738e-209,1.517750e+66,1.606185e+46
300,1.606185e+46,-2.911695e-210,1.517750e+66,1.606185e+46
400,1.606185e+46,-5.208092e-211,1.517750e+66,1.606185e+46
500,1.606185e+46,-1.369363e-211,1.517750e+66,1.606185e+46
600,1.606185e+46,-4.595132e-212,1.517750e+66,1.606185e+46
700,1.606185e+46,-1.824890e-212,1.517750e+66,1.606185e+46
800,1.606185e+46,-8.198790e-213,1.517750e+66,1.606185e+46
900,1.606185e+46,-4.047586e-213,1.517750e+66,1.606185e+46


In [42]:
df.drop('temperature', axis=1).plot()

FigureCanvasNbAgg()

In [14]:
print(dict(h5_files[0].attrs)['git_diff'])

diff --git a/config.json b/config.json
index b86da69..c643735 100644
--- a/config.json
+++ b/config.json
@@ -4,7 +4,7 @@
   "well_depth": 1.6540169269599997e-21
  },
  "N": 4096,
- "file_path": "/mnt/hdd/data/gpu_run_debug/data{0:08d}.h5",
+ "file_path": "/mnt/hdd/data/gpu_run_debug2/data{0:08d}.h5",
  "N_iterations": 100,
  "dt": 1e-09,
  "q": 0,
@@ -12,5 +12,5 @@
  "T": 273,
  "box_L": 1e-07,
  "save_every_x_iters": 10,
- "gpu": True
+ "gpu": true
 }
diff --git a/environment.yml b/environment.yml
index f72146a..dc97461 100644
--- a/environment.yml
+++ b/environment.yml
@@ -4,6 +4,7 @@ channels:
 dependencies:
   - python=3.6
   - numba # TODO pin versions
+  - astropy
   - cupy
   - numpy
   - pytest
diff --git a/nbody/initial_conditions.py b/nbody/initial_conditions.py
index 79a7ba1..bc752d4 100644
--- a/nbody/initial_conditions.py
+++ b/nbody/initial_conditions.py
@@ -8,7 +8,7 @@ import numpy as np
 import math
 import git
 
-k_B = 1.380_648_52e-23
+from nbody.constants import k_B
